In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip -q /content/drive/MyDrive/face_age.zip

In [ ]:

import numpy as np
import os
import pandas as pd
import kagglehub
import tensorflow as tf
import cv2
import imghdr
from tensorflow.keras.metrics import Precision, Recall, Accuracy
from matplotlib import pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Add
from tensorflow.keras.layers import Activation, GlobalAveragePooling2D, Dropout

In [ ]:
data_dir = 'face_age'

In [ ]:
tf.random.set_seed(42)

In [ ]:
image_exts = ["jpeg","jpg","bmp","png"]


In [ ]:
!git clone https://github.com/Alialmanea/age-gender-detection-using-opencv-with-python.git
%cd age-gender-detection-using-opencv-with-python

### **Creating Bias (segregating Male from female)**

In [ ]:
import cv2
import os
import shutil

# Load gender classification model
gender_net = cv2.dnn.readNetFromCaffe("deploy_gender.prototxt", "gender_net.caffemodel")
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
GENDER_LIST = ['Male', 'Female']

# Source and target directories
# Updated source_dir to include the relative path
source_dir = "../face_age"
target_dir = "face_age_male"

def detect_gender(img_path):
    img = cv2.imread(img_path)
    if img is None:
        return None
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    for (x, y, w, h) in faces:
        face_img = img[y:y+h, x:x+w]
        blob = cv2.dnn.blobFromImage(face_img, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        gender_net.setInput(blob)
        gender_preds = gender_net.forward()
        return GENDER_LIST[gender_preds[0].argmax()]
    return None

# Create filtered dataset
for age_folder in os.listdir(source_dir):
    age_path = os.path.join(source_dir, age_folder)
    if not os.path.isdir(age_path):
        continue

    for img_name in os.listdir(age_path):
        img_path = os.path.join(age_path, img_name)
        gender = detect_gender(img_path)

        if gender == "Male":
            target_age_path = os.path.join(target_dir, age_folder)
            os.makedirs(target_age_path, exist_ok=True)
            shutil.copy(img_path, os.path.join(target_age_path, img_name))

In [ ]:

target_dir = "face_age_male"

for age_class in os.listdir(target_dir):
    age_folder_path = os.path.join(target_dir, age_class)
    if not os.path.isdir(age_folder_path):
        continue
    print(f"Age class: {age_class}")
    images = os.listdir(age_folder_path)
    for img_name in images[:3]:
        print(" ", os.path.join(age_folder_path, img_name))

In [ ]:
import glob
for ds_store_file in glob.glob(f"{data_dir}/**/.DS_Store", recursive=True):
    os.remove(ds_store_file)

In [ ]:

class_names = sorted(os.listdir(target_dir))
age_labels = [int(name) for name in class_names]  # Convert subdir names to integers

# lookup table for class index → actual age
age_lookup = tf.constant(age_labels, dtype=tf.int32)

# 3. Map dataset labels to actual ages
data = tf.keras.utils.image_dataset_from_directory(target_dir)
data = data.map(lambda x, y: (x, tf.gather(age_lookup, y)))

# 4. Now apply your age-to-category mapping
def label_to_category(image, label):
    category = tf.where(
        label < 13, 0,
        tf.where(
            label < 20, 1,
            tf.where(label < 60, 2, 3)
        )
    )
    return image, tf.cast(category, tf.int32)

dataset = data.map(label_to_category)


In [ ]:
data_dir = '/content/face_age'
class_names = sorted(os.listdir(data_dir))
age_labels = [int(name) for name in class_names]  # Convert subdir names to integers

# lookup table for class index → actual age
age_lookup = tf.constant(age_labels, dtype=tf.int32)

# 3. Map dataset labels to actual ages
data = tf.keras.utils.image_dataset_from_directory(data_dir)
data = data.map(lambda x, y: (x, tf.gather(age_lookup, y)))

# 4. Now apply your age-to-category mapping
def label_to_category(image, label):
    category = tf.where(
        label < 13, 0,
        tf.where(
            label < 20, 1,
            tf.where(label < 60, 2, 3)
        )
    )
    return image, tf.cast(category, tf.int32)

dataset_general = data.map(label_to_category)


In [ ]:
for image, label in dataset.take(1):
    print("Image shape:", image.shape)
    print("Label:", label)

In [ ]:
dataset = dataset.map(lambda x,y:(x/255,y))

In [ ]:
dataset_general = dataset_general.map(lambda x,y:(x/255,y))

In [ ]:
scaled_iterator_1 = dataset.as_numpy_iterator()

In [ ]:
batch_dataset = scaled_iterator_1.next()

In [ ]:
scaled_iterator_2 = dataset_general.as_numpy_iterator()

In [ ]:
fix , ax = plt.subplots(ncols = 4,figsize = (20,20))
for indx,img in enumerate(batch_dataset[0][:4]):
    ax[indx].imshow((img * 255).astype("uint8"))
    ax[indx].title.set_text(f"Label: {batch_dataset[1][indx]}") # Changed batch to batch_dataset

In [ ]:
length = dataset.cardinality().numpy()

train_size_d = int(length * 0.8)
val_size_d = int(length * 0.2)

train_dataset = dataset.take(train_size_d)
val_dataset = dataset.skip(train_size_d).take(val_size_d)

In [ ]:
train_dataset_general = dataset_general.take(train_size_d)
val_dataset_general = dataset_general.skip(train_size_d).take(val_size_d)



In [ ]:
test_size_d = int(length * 0.1)
test_dataset = dataset_general.skip(train_size_d + val_size_d).take(test_size_d)


## **DeepLearing MODEL(CLassification)**

In [ ]:
def build_simple_cnn(
    input_shape=(256, 256, 3),
    hidden_layers=2,
    filters=[32, 16],
    kernel_size=(3, 3),
    activation='relu',
    include_pooling=True,
    fc_layers=[256],  # list for FC layers: e.g., [256, 128]
    output_classes=4,
    use_skip_connections=False
):
    inputs = Input(shape=input_shape)
    x = Conv2D(16, kernel_size, padding='same', activation=activation)(inputs)
    if include_pooling:
        x = MaxPooling2D()(x)

    prev = x  # to store for skip connection

    for i in range(hidden_layers * 2):
        filter_idx = i % len(filters)
        current_filter = filters[filter_idx]

        conv = Conv2D(current_filter, kernel_size, padding='same', activation=activation)(x)

        if include_pooling:
            conv = MaxPooling2D()(conv)

        if use_skip_connections and conv.shape == prev.shape:
            x = Add()([conv, prev])  # skip connection
        else:
            x = conv

        prev = x

    # Flatten or GlobalAvgPool
    x = Flatten()(x)  # or use GlobalAveragePooling2D()

    # Fully Connected Layers
    for fc_size in fc_layers:
        x = Dense(fc_size, activation=activation)(x)

    outputs = Dense(output_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model



model_1 = build_simple_cnn(activation='tanh')

model_2 = build_simple_cnn(activation='tanh')

### **Training the model with only one gender(MALE)**

In [ ]:
hist_1 = model_1.fit(train_dataset,epochs = 20,validation_data = val_dataset)

### **Training the model with Both Genders**

In [ ]:
hist_2 = model_2.fit(train_dataset_general,epochs = 20,validation_data = val_dataset_general)

In [ ]:
def evaluate_classification_model(model, test_dataset):
    pre = Precision()
    re = Recall()
    acc = Accuracy()

    for batch in test_dataset.as_numpy_iterator():
        x, y = batch
        yhat = model.predict(x, verbose=0)

        # Get true and predicted class indices
        y_true = y.argmax(axis=1) if y.ndim > 1 else y
        y_pred = yhat.argmax(axis=1)

        # Update metrics
        pre.update_state(y_true, y_pred)
        re.update_state(y_true, y_pred)
        acc.update_state(y_true, y_pred)

    precision = pre.result().numpy()
    recall = re.result().numpy()
    accuracy = acc.result().numpy()
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-8)  # Avoid division by zero

    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1-score: {f1_score:.4f}")

    return {
        "precision": precision,
        "recall": recall,
        "accuracy": accuracy,
        "f1_score": f1_score
    }


In [ ]:

model_1_test = evaluate_classification_model(model_1, test_dataset)

In [ ]:
model_2_test = evaluate_classification_model(model_2, test_dataset)

In [ ]:
from tensorflow import keras

# Save regression model
model_1.save('gender_model_male.keras')

# Save classification model
model_2.save('gender_model_general.keras')


In [ ]:
import os
import shutil

os.makedirs("part_3_models", exist_ok=True)
shutil.move("gender_model_male.keras", "part_3_models/gender_model_male.keras")
shutil.move("gender_model_general.keras", "part_3_models/gender_model_general.keras")
shutil.make_archive("part_3_models", 'zip', "part_3_models")


In [ ]:

from google.colab import files
files.download('/content/age-gender-detection-using-opencv-with-python/part_3_models.zip')
